<a href="https://colab.research.google.com/github/nakap21/Raiffeisen/blob/master/HomeCredit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nakap21","key":"4d1452c79287f7113d5d22b1ca7418c9"}'}

In [2]:
!pip install -q kaggle
!pip install -q catboost

     |████████████████████████████████| 66.1MB 59kB/s 


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle competitions download -c home-credit-default-risk

  0% 0.00/524k [00:00<?, ?B/s]
100% 524k/524k [00:00<00:00, 87.0MB/s]
 90% 33.0M/36.8M [00:00<00:00, 28.9MB/s]
100% 36.8M/36.8M [00:01<00:00, 37.9MB/s]
 96% 260M/271M [00:07<00:00, 30.9MB/s]
100% 271M/271M [00:07<00:00, 37.6MB/s]
 92% 89.0M/96.7M [00:01<00:00, 53.3MB/s]
100% 96.7M/96.7M [00:01<00:00, 71.6MB/s]
 64% 49.0M/76.3M [00:00<00:00, 31.3MB/s]
100% 76.3M/76.3M [00:01<00:00, 78.7MB/s]
 86% 5.00M/5.81M [00:00<00:00, 32.9MB/s]
100% 5.81M/5.81M [00:00<00:00, 37.1MB/s]
 91% 33.0M/36.1M [00:00<00:00, 56.5MB/s]
100% 36.1M/36.1M [00:00<00:00, 81.4MB/s]
 98% 106M/109M [00:02<00:00, 31.9MB/s] 
100% 109M/109M [00:02<00:00, 53.0MB/s]
 88% 50.0M/56.8M [00:01<00:00, 18.7MB/s]
100% 56.8M/56.8M [00:01<00:00, 48.9MB/s]
  0% 0.00/36.5k [00:00<?, ?B/s]
100% 36.5k/36.5k [00:00<00:00, 37.8MB/s]


In [11]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

In [6]:
train = pd.read_csv('application_train.csv.zip', compression='zip')

train_numeric = train.select_dtypes([np.number]).fillna(0)

X = train_numeric.drop(labels=['TARGET'], axis="columns")
y = train_numeric['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# reg = LogisticRegression(max_iter=1000000).fit(X_train, y_train)
gb = XGBClassifier(tree_method='gpu_hist').fit(X_train, y_train)

In [12]:
print(accuracy_score(y_test, gb.predict(X_test)))
print(precision_score(y_test, gb.predict(X_test)))
print(recall_score(y_test, gb.predict(X_test)))
print(roc_auc_score(y_test, gb.predict_proba(X_test)[:, 1]))

0.9195811586426679
0.56
0.002828854314002829
0.7436057604734183


## bureau

In [13]:
bureau = pd.read_csv('bureau.csv.zip', compression='zip')
data = X

In [14]:
# Считаем активные и закрытые кредиты.
cnt_active_loans = bureau[bureau.CREDIT_ACTIVE == 'Active'].groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE'].count()
cnt_closed_loans = bureau[bureau.CREDIT_ACTIVE == 'Closed'].groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE'].count()

In [15]:
# Мерджим фичи

data = data.merge(cnt_active_loans, on = 'SK_ID_CURR', how = 'left').fillna(0)
data = data.merge(cnt_closed_loans, on = 'SK_ID_CURR', how = 'left').fillna(0)

In [16]:
# Теперь посчитаем для всех числовых колонок статистику по ним, что так же будет
# являтся хорошими фичами для нашей модели.

cnt_numeric_fields_statistics = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False).agg(['min', 'max', 'mean', 'sum'])
cnt_numeric_fields_statistics.reset_index(inplace=True)

In [17]:
columns = ['SK_ID_CURR']
for column in cnt_numeric_fields_statistics.columns.levels[0]:
    if column != 'SK_ID_CURR':
        for stat in cnt_numeric_fields_statistics.columns.levels[1][:-1]:
            columns.append('bureau_%s_%s' % (column, stat))

cnt_numeric_fields_statistics.columns = columns

In [18]:
data = data.merge(cnt_numeric_fields_statistics, on = 'SK_ID_CURR', how = 'left').fillna(0)

In [19]:
# Оставшиеся текстовые колонки ohe кодируем, считая nan отдельным классом.

def one_hot_encoder(df):
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    return pd.get_dummies(df, columns=categorical_columns, dummy_na=True)

In [20]:

credit_type_ohe = one_hot_encoder(bureau)

credit_type_ohe = credit_type_ohe.groupby('SK_ID_CURR', as_index=False).count()

data = data.merge(credit_type_ohe, on = 'SK_ID_CURR', how = 'left').fillna(0)

## **SCORE**

In [21]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

In [22]:
gb_new = XGBClassifier(tree_method='gpu_hist').fit(X_train, y_train)

In [23]:
print(accuracy_score(y_test, gb_new.predict(X_test)))
print(precision_score(y_test, gb_new.predict(X_test)))
print(recall_score(y_test, gb_new.predict(X_test)))
print(roc_auc_score(y_test, gb_new.predict_proba(X_test)[:, 1]))

0.9195974180121295
0.5909090909090909
0.0026267932915740554
0.7468790050494389


## **МОИ ФИЧИ**

In [26]:
IP = pd.read_csv('installments_payments.csv.zip', compression='zip')
BadSKID = list(IP[IP['DAYS_ENTRY_PAYMENT'].isna()]['SK_ID_CURR'])
new = IP[['SK_ID_CURR', 'AMT_INSTALMENT', 'AMT_PAYMENT', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT']]

def f(x):
  if x['SK_ID_CURR'] in BadSKID:
    x['IN_BLACK_LIST'] = 'bad'
  return x


del IP
new = new.dropna()
new = new.groupby(['SK_ID_CURR'], as_index=False).sum()
new['IN_BLACK_LIST'] = 'good'
new['DAYS_GONE'] = new['DAYS_ENTRY_PAYMENT'] - new['DAYS_INSTALMENT']
new = new.apply(f, axis=1)
new = new.drop(columns=['DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT'])

In [27]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339578 entries, 0 to 339577
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SK_ID_CURR      339578 non-null  int64  
 1   AMT_INSTALMENT  339578 non-null  float64
 2   AMT_PAYMENT     339578 non-null  float64
 3   IN_BLACK_LIST   339578 non-null  object 
 4   DAYS_GONE       339578 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 15.5+ MB


In [28]:
credit_card_balance = pd.read_csv('credit_card_balance.csv.zip', compression='zip')

cc = credit_card_balance[['SK_ID_CURR', 'SK_DPD', 'SK_DPD_DEF']]
cc = cc.groupby(['SK_ID_CURR'], as_index=False).sum()

In [30]:
new_ohe = one_hot_encoder(new)

In [31]:
values = {'SK_DPD': 0, 'SK_DPD_DEF': 0, 'AMT_INSTALMENT': 0, 'AMT_PAYMENT': 0, 'DAYS_GONE': 0}
data = pd.merge(data, cc, how='left', on='SK_ID_CURR')
data = pd.merge(data, new_ohe, how='left', on='SK_ID_CURR')
data = data.fillna(value=values)

# END

In [35]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

In [36]:
gb_new = XGBClassifier(tree_method='gpu_hist').fit(X_train, y_train)

In [37]:
print(accuracy_score(y_test, gb_new.predict(X_test)))
print(precision_score(y_test, gb_new.predict(X_test)))
print(recall_score(y_test, gb_new.predict(X_test)))
print(roc_auc_score(y_test, gb_new.predict_proba(X_test)[:, 1]))

0.9195974180121295
0.5833333333333334
0.002828854314002829
0.7483946717315143
